# Particle based reaction diffusion simulations with ReaDDy
In this ipython notebook we will go over the principal workflow of ReaDDy, which is:
1. Create and configure a `readdy.ReactionDiffusionSystem`, i.e. particle species, potentials, reactions, topologies
2. Get a `Simulation` object and configure it, i.e. output file, observables, initial particle positions
3. Run the `Simulation`, by choosing a time step size and the number of integration steps
4. Analyze results

Keep in mind:
- When in doubt, refer to the online documentation https://readdy.github.io !
- Make use of tab-completion in notebooks !

In [2]:
import os
import readdy
print(readdy.__version__)

v1.0.1-68


## One diffusing particle

Create and configure a system

In [3]:
system = readdy.ReactionDiffusionSystem(box_size=[10.,10.,10.], unit_system=None)

In [4]:
system.add_species("A", diffusion_constant=1.0)

Get a simulation object and configure output

In [5]:
simulation = system.simulation(kernel="SingleCPU")

In [6]:
simulation.output_file = "./oneparticle.h5"
simulation.record_trajectory()

In [7]:
simulation.add_particle("A", [0.,0.,0.])

Run the simulation

In [8]:
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)

simulation.run(n_steps=1000, timestep=0.1)

Configured kernel context with:
--------------------------------
 - kBT = 1
 - periodic b.c. = (true, true, true)
 - box size = (10, 10, 10)
 - particle types:
     *  particle type "A" with D=1



[2018-07-16 14:06:14] [info] Simulation completed


Look at results. The `!` prefix in a cell forwards the command to the (bash) shell.

In [9]:
!ls | grep oneparticle

oneparticle.h5


In [10]:
traj = readdy.Trajectory(filename="oneparticle.h5")

In [11]:
traj.convert_to_xyz()

In [12]:
!ls | grep oneparticle

oneparticle.h5
oneparticle.h5.xyz
oneparticle.h5.xyz.tcl


In [13]:
!vmd -e oneparticle.h5.xyz.tcl > /dev/null

/home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64: /usr/lib/x86_64-linux-gnu/mesa/libGL.so.1: no version information available (required by /home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64)


## Two species

In [14]:
system = readdy.ReactionDiffusionSystem([10.,10.,10.], unit_system=None)
system.add_species("A", diffusion_constant=1.0)
system.add_species("B", diffusion_constant=0.1)

In [15]:
simulation = system.simulation("SingleCPU")
simulation.output_file = "twospecies.h5"
simulation.record_trajectory()
simulation.add_particle("A", [0.,0.,0.])
simulation.add_particle("B", [1.,1.,1.])

In [16]:
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)

simulation.run(1000, 0.1)

Configured kernel context with:
--------------------------------
 - kBT = 1
 - periodic b.c. = (true, true, true)
 - box size = (10, 10, 10)
 - particle types:
     *  particle type "B" with D=0.1
     *  particle type "A" with D=1



[2018-07-16 14:06:19] [info] Simulation completed


In [17]:
!ls | grep twospecies

twospecies.h5


In [18]:
traj = readdy.Trajectory("twospecies.h5")
traj.convert_to_xyz()

In [19]:
!vmd -e twospecies.h5.xyz.tcl > /dev/null

/home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64: /usr/lib/x86_64-linux-gnu/mesa/libGL.so.1: no version information available (required by /home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64)


## Reactions, Fusion
see https://readdy.github.io/system.html#reactions

In [20]:
system = readdy.ReactionDiffusionSystem([10.,10.,10.], unit_system=None)
system.add_species("A", diffusion_constant=1.0)
system.add_species("B", diffusion_constant=1.0)
system.add_species("C", diffusion_constant=0.01)
system.reactions.add("foo: A +(3.0) B -> C", rate=0.5)
# or equivalently
# system.reactions.add_fusion("foo", "A", "B", "C", 0.5, 3.0)

In [21]:
simulation = system.simulation("SingleCPU")
simulation.output_file = "reactions.h5"
simulation.record_trajectory()
simulation.add_particle("A", [-2.,0.,0.])
simulation.add_particle("B", [+2.,0.,0.])

In [22]:
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)

simulation.run(1000, 0.01)

Configured kernel context with:
--------------------------------
 - kBT = 1
 - periodic b.c. = (true, true, true)
 - box size = (10, 10, 10)
 - particle types:
     *  particle type "C" with D=0.01
     *  particle type "A" with D=1
     *  particle type "B" with D=1
 - bimolecular reactions:
     * Fusion A + B -> C with a rate of 0.5, an educt distance of 3, and weights 0.5 and 0.5



[2018-07-16 14:06:22] [info] Simulation completed


In [23]:
!ls | grep reactions

reactions.h5


We can assign radii for the particle species for the visualization. Since the reaction radius is 3, it makes sense to assign A and B a radius of 1.5 each.

In [24]:
traj = readdy.Trajectory("reactions.h5")
traj.convert_to_xyz(particle_radii={"A": 1.5, "B": 1.5, "C": 1.7})

In [25]:
!ls | grep reactions

reactions.h5
reactions.h5.xyz
reactions.h5.xyz.tcl


In [26]:
!vmd -e reactions.h5.xyz.tcl > /dev/null

/home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64: /usr/lib/x86_64-linux-gnu/mesa/libGL.so.1: no version information available (required by /home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64)


## Reactions, Fusion and Fission

In [27]:
system = readdy.ReactionDiffusionSystem([10.,10.,10.], unit_system=None)
system.add_species("A", 1.0)
system.add_species("B", 1.0)
system.add_species("C", 0.01)
system.reactions.add("foo: A +(3.0) B -> C", rate=0.5)
system.reactions.add("bar: C -> A +(3.0) B ", rate=0.5)

In [28]:
simulation = system.simulation("SingleCPU")
simulation.output_file = "reactions2.h5"
simulation.record_trajectory()

This time we'll add many particles

In [29]:
import numpy as np

In [30]:
N = 20
extent = 10.
origin = -5.
positions_a = np.random.uniform(size=(N,3)) * extent + origin
positions_b = np.random.uniform(size=(N,3)) * extent + origin

In [31]:
simulation.add_particles("A", positions_a)
simulation.add_particles("B", positions_b)

In [32]:
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)
simulation.run(1000, 0.01)

Configured kernel context with:
--------------------------------
 - kBT = 1
 - periodic b.c. = (true, true, true)
 - box size = (10, 10, 10)
 - particle types:
     *  particle type "C" with D=0.01
     *  particle type "A" with D=1
     *  particle type "B" with D=1
 - unimolecular reactions:
     * Fission C -> A + B with a rate of 0.5, a product distance of 3, and weights 0.5 and 0.5
 - bimolecular reactions:
     * Fusion A + B -> C with a rate of 0.5, an educt distance of 3, and weights 0.5 and 0.5



[2018-07-16 14:06:26] [info] Simulation completed


In [33]:
traj = readdy.Trajectory("reactions2.h5")
traj.convert_to_xyz(particle_radii={"A": 1.5, "B": 1.5, "C": 1.7})

In [34]:
!vmd -e reactions2.h5.xyz.tcl > /dev/null

/home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64: /usr/lib/x86_64-linux-gnu/mesa/libGL.so.1: no version information available (required by /home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64)


## Potentials, box
see https://readdy.github.io/system.html#box-potential

In [35]:
system = readdy.ReactionDiffusionSystem([10.,10.,10.], unit_system=None)
system.add_species("A", 1.)
system.add_species("B", 1.)

In [36]:
system.potentials.add_box("A", force_constant=20., origin=[-4.,-4.,-4.], extent=[8.,8.,8.])

system.potentials.add_box("B", force_constant=30., origin=[-4.,-4.,-4.], extent=[0.1,8.,8.])

In [37]:
simulation = system.simulation("SingleCPU")
simulation.output_file = "box.h5"
simulation.record_trajectory()
simulation.add_particles("A", np.zeros((40,3)))
simulation.add_particles("B", np.zeros((40,3)))

In [38]:
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)
simulation.run(10000, 0.001)

Configured kernel context with:
--------------------------------
 - kBT = 1
 - periodic b.c. = (true, true, true)
 - box size = (10, 10, 10)
 - particle types:
     *  particle type "B" with D=1
     *  particle type "A" with D=1
 - potentials of order 1:
     * for type "B"
         * Box potential with origin=(-4, -4, -4), extent=(0.1, 8, 8), and force constant k=30
     * for type "A"
         * Box potential with origin=(-4, -4, -4), extent=(8, 8, 8), and force constant k=20



[2018-07-16 14:06:33] [info] Simulation completed


In [39]:
traj = readdy.Trajectory("box.h5")
traj.convert_to_xyz()

In [40]:
!vmd -e box.h5.xyz.tcl > /dev/null

/home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64: /usr/lib/x86_64-linux-gnu/mesa/libGL.so.1: no version information available (required by /home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64)


## Potentials, pair-wise repulsion

In [42]:
system = readdy.ReactionDiffusionSystem([10.,10.,10.], unit_system=None)
system.add_species("A", 0.1)
system.potentials.add_harmonic_repulsion("A", "A", force_constant=100., interaction_distance=2.)
simulation = system.simulation("SingleCPU")
simulation.output_file = "repulsion.h5"
simulation.record_trajectory()
simulation.add_particles("A", np.random.uniform(size=(30, 3)) * 10. - 5.)
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)
simulation.run(10000, 0.01)
traj = readdy.Trajectory("repulsion.h5")
traj.convert_to_xyz()

Configured kernel context with:
--------------------------------
 - kBT = 1
 - periodic b.c. = (true, true, true)
 - box size = (10, 10, 10)
 - particle types:
     *  particle type "A" with D=0.1
 - potentials of order 2:
     * for types "A" and "A"
         * Harmonic repulsion with force constant k=100



[2018-07-16 14:12:38] [info] Simulation completed


In [43]:
!vmd -e repulsion.h5.xyz.tcl > /dev/null

/home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64: /usr/lib/x86_64-linux-gnu/mesa/libGL.so.1: no version information available (required by /home/chris/misc/vmd/lib/vmd/vmd_LINUXAMD64)


It is hard to tell from the visualization if particles actually repulse each other. Let's make use of observables.

In [48]:
system = readdy.ReactionDiffusionSystem([20.,20.,20.], unit_system=None)
system.add_species("A", 1.)
system.add_species("B", 1.)
system.potentials.add_harmonic_repulsion("A", "A", force_constant=100., interaction_distance=2.)

In [49]:
simulation = system.simulation("SingleCPU")
simulation.output_file = "rdf.h5"
n_particles = 100
# factor of 2 in particle_to_density, because in rdf from A->A every distance is seen twice
simulation.observe.rdf(stride=100, bin_borders=np.arange(0.,10.,0.2),
                       types_count_from="A", types_count_to="A",
                       particle_to_density=2.*n_particles/system.box_volume,
                       save={"name": "rdfa", "chunk_size": 1000})
simulation.observe.rdf(stride=100, bin_borders=np.arange(0.,10.,0.2),
                       types_count_from="B", types_count_to="B",
                       particle_to_density=2.*n_particles/system.box_volume,
                       save={"name": "rdfb", "chunk_size": 1000})

simulation.add_particles("A", np.random.uniform(size=(n_particles, 3)) * 20. - 10.)
simulation.add_particles("B", np.random.uniform(size=(n_particles, 3)) * 20. - 10.)
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)
simulation.run(100000, 0.01)
traj = readdy.Trajectory("rdf.h5")


Configured kernel context with:
--------------------------------
 - kBT = 1
 - periodic b.c. = (true, true, true)
 - box size = (20, 20, 20)
 - particle types:
     *  particle type "B" with D=1
     *  particle type "A" with D=1
 - potentials of order 2:
     * for types "A" and "A"
         * Harmonic repulsion with force constant k=100



[2018-07-16 17:23:26] [info] Simulation completed
